In [1]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains
# import pandas as pd
# import time
# from dotenv import load_dotenv
# import os
# import openai

# from bs4 import BeautifulSoup
# import re
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# import time
# import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from dotenv import load_dotenv
import os
import openai

from bs4 import BeautifulSoup
import re
import time
import pickle


In [2]:


def find_linkedin_info(driver, profile_url):
    driver.get(profile_url)
    time.sleep(5)  # Allow the page to load

    # Get the source HTML of the page
    source = driver.page_source

    # Parse the source HTML with BeautifulSoup
    soup = BeautifulSoup(source, 'lxml')

    # Get the text from the parsed HTML
    page_text = soup.get_text()
    
    # Split the text into lines and remove empty lines
    lines = page_text.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != ""]

    # Join the non-empty lines back into a single string
    url_text = "\n".join(non_empty_lines)

    return url_text


In [3]:

import re

class Extractor:
    '''
    
To request an appropriate pattern or string match for this class, you could ask:

"Please provide a string or a regular expression pattern that we should use for the start 
rule or end rule. If you provide a regular expression pattern, please specify that it is 
a regex. Also, note that for regular expressions, we're using Python's 're' module, so the 
pattern should be compatible with it. If you want to extract from the start or end of the 
text when no matching rule is found, please indicate that as well."
    '''

    def __init__(self):
        self.start_rules = []
        self.end_rules = []

    def add_start_rule(self, rule, is_regex=False):
        self.start_rules.append((rule, is_regex))

    def add_end_rule(self, rule, is_regex=False):
        self.end_rules.append((rule, is_regex))

    def extract(self, text, extract_if_no_start=False, extract_if_no_end=False):
        if len(self.start_rules) > 0 and not extract_if_no_start:
            start_index = None
        else:
            start_index = 0

        if len(self.end_rules) > 0 and not extract_if_no_end:
            end_index = None
        else:
            end_index = len(text)



        for rule, is_regex in self.start_rules:
            if is_regex:
                match = re.search(rule, text)
                if match is not None:
                    start_index = match.end()  # We want the index after the start rule
                    break  # If we've found a match, we can break
            else:
                idx = text.find(rule)
                if idx != -1:
                    start_index = idx + len(rule)  # We want the index after the start rule
                    break  # If we've found a match, we can break

        for rule, is_regex in self.end_rules:
            if is_regex:
                match = re.search(rule, text[start_index if start_index is not None else 0:])
                if match is not None:
                    end_index = (start_index if start_index is not None else 0) + match.start()  # We want the index before the end rule
                    break  # If we've found a match, we can break
            else:
                idx = text.find(rule, start_index if start_index is not None else 0)  # We search after the start index
                if idx != -1:
                    end_index = idx
                    break  # If we've found a match, we can break

        if start_index is None or end_index is None:
            return ''
        
        return text[start_index:end_index]





In [4]:

# def string_parcer_search_page(s):
#     pattern = r"(?<=search result pages)(.*?)(?=Page \d+ of \d+)"
#     match = re.search(pattern, s, re.DOTALL)
#     if match:
#         print("Match found!")
#         string_to_GPT = match.group().strip()
#         return string_to_GPT
#     else:
#         return None



In [5]:

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=chrome_options)

driver.get('https://www.linkedin.com/')

# Load cookies if they exist
try:
    cookies = pickle.load(open("linkedin_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()
except:
    print("No cookies found. Manual login required.")

# Check if login is needed
try: # not working
    element_present = EC.presence_of_element_located((By.ID, 'profile-nav-item')) # check for an element that is only present when logged in
    WebDriverWait(driver, 10).until(element_present)
except:
    # If not logged in
    input('Please login and press Enter to continue...')
    pickle.dump(driver.get_cookies(), open("linkedin_cookies.pkl", "wb")) # save cookies after login


# driver.quit()


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


Please login and press Enter to continue...


In [6]:
instructions = """
Extract the names: Please look for patterns indicating a person's name. Names are usually proper nouns and are generally the first information provided in each data point.

Find the job titles: Please identify each person's job title. This information follows the name and often contains phrases such as 'Data Scientist', 'Neuroscientist', 'Senior Associate Data Scientist', or a variant that includes the role and any additional qualifiers.

Identify the job location: This information typically follows the job title and is usually a city or metropolitan area, such as 'Pasadena, CA' or 'Los Angeles Metropolitan Area'.

Find the company names: The company names usually follow the job location. In case there is no current company mentioned, or the person is actively seeking a job, make a note of that by using placeholders like 'None (Job Seeking)' or 'None (No Company Mentioned)'.

Construct the dictionary: Organize all the extracted information into a dictionary format where the key represents the type of information (e.g., 'Name', 'Job Titles', 'Job Location', 'Company Name') and the value is a list of all the extracted information of that type in the order of appearance.

Formatting the instructions: Always make sure to start the code block with a comment '##BEGIN##', fill in the data accordingly, and end it with '##END##'. All columns of data should align with each other.

For example:


##BEGIN##

DF_TEMP = {
    "Name": ["Person 1", "Person 2", ...],
    "Job Titles": ["Job 1", "Job 2", ...],
    "Job Location": ["Location 1", "Location 2", ...],
    "Company Name": ["Company 1", "Company 2", ...]
}

##END##


Replace "Person 1", "Job 1", "Location 1", "Company 1", etc., with the actual extracted data.

Please make sure to follow these instructions to achieve consistent results.


here is the text:



"""

In [7]:

# def string_parcer_search_page(s):
#     pattern = r"(?<=search result pages)(.*?)(?=Page \d+ of \d+)"
#     match = re.search(pattern, s, re.DOTALL)
#     if match:
#         print("Match found!")
#         string_to_GPT = match.group().strip()
#         return string_to_GPT
#     else:
#         return None



In [8]:
# url_text = find_linkedin_info(driver, url_in)
# text_finder = Extractor()
# text_finder.add_start_rule('search result pages', False)
# text_finder.add_end_rule('Page \d+ of \d+', True)
# text_finder.add_end_rule("messaging overlay", False)
# out = text_finder.extract(url_text)
# print(out)


In [9]:
url_in = 'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'




In [10]:
url_text = find_linkedin_info(driver, url_in)
text_finder = Extractor()
text_finder.add_start_rule('search result pages', False)
text_finder.add_end_rule('Page \d+ of \d+', True)
text_finder.add_end_rule("these results helpful", False)
text_finder.add_end_rule("messaging overlay", False)

gpt_text = text_finder.extract(url_text)
GPT_INPUT = instructions+gpt_text

In [11]:
print(GPT_INPUT)


Extract the names: Please look for patterns indicating a person's name. Names are usually proper nouns and are generally the first information provided in each data point.

Find the job titles: Please identify each person's job title. This information follows the name and often contains phrases such as 'Data Scientist', 'Neuroscientist', 'Senior Associate Data Scientist', or a variant that includes the role and any additional qualifiers.

Identify the job location: This information typically follows the job title and is usually a city or metropolitan area, such as 'Pasadena, CA' or 'Los Angeles Metropolitan Area'.

Find the company names: The company names usually follow the job location. In case there is no current company mentioned, or the person is actively seeking a job, make a note of that by using placeholders like 'None (Job Seeking)' or 'None (No Company Mentioned)'.

Construct the dictionary: Organize all the extracted information into a dictionary format where the key repres

In [12]:
# from bs4 import BeautifulSoup


# # driver.get(profile_url)
# # time.sleep(5)  # Allow the page to load

# # Get the source HTML of the page
# source = driver.page_source

# # Parse the source HTML with BeautifulSoup
# soup = BeautifulSoup(source, 'lxml')

# # Get the text from the parsed HTML
# page_text = soup.get_text()
# print(page_text)

In [13]:
load_dotenv()  # take environment variables from .env.
os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")
openai.api_key = os.environ["OPENAI_API_KEY"]


In [14]:


messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]
messages.append({"role": "user", "content": GPT_INPUT})
chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
reply = chat.choices[0].message.content


In [15]:
print(reply)

##BEGIN##

DF_TEMP = {
    "Name": ["Abhinav R.", "Shijie (Selene) Xiang", "Jinze Xin", "Syamil Mohd Razak, PhD", "Justin Chen", "Malavika Ajith Nair", "Marshall R.", "Kevin Goldberg", "Hayden Thornton", "Lu L."],
    "Job Titles": ["Incoming Data Scientist at Chevron | USC Alum", "Data Scientist at eBay", "Data Scientist at Twitter", "Data Scientist | Petroleum Engineer", "Data Scientist at Snap Inc.", "Data Scientist", "Data Scientist | web3/DeFi", "Data Scientist at Meta", "Data Scientist at McKinsey & Company", "Data Scientist at Facebook"],
    "Job Location": ["Los Angeles, CA", "Los Angeles, CA", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "United States", "United States", "Los Angeles, CA", "Los Angeles, CA", "Los Angeles, CA"],
    "Company Name": ["Chevron", "eBay", "Twitter", "Phillips 66", "Snap Inc.", "Meta", "Stealth Startup", "Meta Reality Labs", "McKinsey & Company", "Facebook"]
}

##END##


In [16]:
import pandas as pd

# Extracted information from the text
##BEGIN##

DF_TEMP = {
    "Name": ["Abhinav R.", "Jinze Xin", "Syamil Mohd Razak, PhD", "Shijie (Selene) Xiang", "Malavika Ajith Nair", "Marshall R.", "Mehrnaz Motamed", "Farimah Shirmohammadi, PhD", "Kevin Goldberg", "Lu L."],
    "Job Titles": ["Incoming Data Scientist at Chevron | USC Alum", "Data Scientist at Twitter", "Data Scientist | Petroleum Engineer", "Data Scientist at eBay", "Data Scientist", "Data Scientist | web3/DeFi", "Data Scientist - Machine Learning | Deep Learning | Statistics | R | SQL | Python", "Data Scientist", "Data Scientist at Meta", "Data Scientist at Facebook"],
    "Job Location": ["Los Angeles, CA", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "Los Angeles, CA", "United States", "United States", "Pasadena, CA", "Los Angeles, CA", "Los Angeles, CA", "Los Angeles, CA"],
    "Company Name": ["Chevron", "Twitter", "Phillips 66", "eBay", "Meta", "Stealth Startup", "Endura Technologies", "Edison International", "Meta", "Facebook"]
}


# Create a DataFrame
DF_TEMP = pd.DataFrame(DF_TEMP)
DF_TEMP



,Name,Job Titles,Job Location,Company Name
0,Abhinav R.,Incoming Data Scientist at Chevron | USC Alum,"Los Angeles, CA",Chevron
1,Jinze Xin,Data Scientist at Twitter,Los Angeles Metropolitan Area,Twitter
2,"Syamil Mohd Razak, PhD",Data Scientist | Petroleum Engineer,Los Angeles Metropolitan Area,Phillips 66
3,Shijie (Selene) Xiang,Data Scientist at eBay,"Los Angeles, CA",eBay
4,Malavika Ajith Nair,Data Scientist,United States,Meta
5,Marshall R.,Data Scientist | web3/DeFi,United States,Stealth Startup
6,Mehrnaz Motamed,Data Scientist - Machine Learning | Deep Learn...,"Pasadena, CA",Endura Technologies
7,"Farimah Shirmohammadi, PhD",Data Scientist,"Los Angeles, CA",Edison International
8,Kevin Goldberg,Data Scientist at Meta,"Los Angeles, CA",Meta
9,Lu L.,Data Scientist at Facebook,"Los Angeles, CA",Facebook


In [17]:
DF_TEMP

,Name,Job Titles,Job Location,Company Name
0,Abhinav R.,Incoming Data Scientist at Chevron | USC Alum,"Los Angeles, CA",Chevron
1,Jinze Xin,Data Scientist at Twitter,Los Angeles Metropolitan Area,Twitter
2,"Syamil Mohd Razak, PhD",Data Scientist | Petroleum Engineer,Los Angeles Metropolitan Area,Phillips 66
3,Shijie (Selene) Xiang,Data Scientist at eBay,"Los Angeles, CA",eBay
4,Malavika Ajith Nair,Data Scientist,United States,Meta
5,Marshall R.,Data Scientist | web3/DeFi,United States,Stealth Startup
6,Mehrnaz Motamed,Data Scientist - Machine Learning | Deep Learn...,"Pasadena, CA",Endura Technologies
7,"Farimah Shirmohammadi, PhD",Data Scientist,"Los Angeles, CA",Edison International
8,Kevin Goldberg,Data Scientist at Meta,"Los Angeles, CA",Meta
9,Lu L.,Data Scientist at Facebook,"Los Angeles, CA",Facebook


In [ ]:
# import openai
# openai.api_key = 'YOUR_API_KEY'
messages = [ {"role": "system", "content": 
              "You are a intelligent assistant."} ]
while True:
    message = input("User : ")
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        asdf
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    reply = chat.choices[0].message.content
    print(f"ChatGPT: {reply}")
    messages.append({"role": "assistant", "content": reply})

In [ ]:
# Split the text into lines and remove empty lines
lines = page_text.split('\n')
non_empty_lines = [line for line in lines if line.strip() != ""]

# Join the non-empty lines back into a single string
text_without_empty_lines = "\n".join(non_empty_lines)
print(text_without_empty_lines)

In [ ]:
import re

# The string to search in
s = text_without_empty_lines

# The regex pattern with groups
pattern = r"(?<=search result pages)(.*?)(?=Page \d+ of \d+)"

s# Search for the pattern
match = re.search(pattern, s, re.DOTALL)

if match:
    print("Match found. Extracted text:")
    string_to_GPT = match.group().strip()
    print(string_to_GPT)
else:
    print("No match found")


In [ ]:
def find_linkedin_info(driver, profile_url):
    driver.get(profile_url)
    time.sleep(5)  # Allow the page to load
    page_text = driver.find_element_by_tag_name('body').text
    return page_text



profile_url = linkedin_urls[0]


driver.get(profile_url)
time.sleep(5)  # Allow the page to load
page_text = driver.find_element(By.TAG_NAME, 'body').text


In [ ]:
print(page_text)

In [ ]:
import re

# The string to search in
s = page_text

# The regex pattern with groups
pattern = r"(?<=search result pages)(.*?)(?=Page \d+ of \d+)"

s# Search for the pattern
match = re.search(pattern, s, re.DOTALL)

if match:
    print("Match found. Extracted text:")
    string_to_GPT = match.group().strip()
    print(string_to_GPT)
else:
    print("No match found")


In [ ]:
print(1)

In [ ]:

# # Load the spreadsheet into a pandas DataFrame
# df = pd.read_excel(sheet_name)

# # Load the Excel workbook using openpyxl
# workbook = load_workbook(sheet_name)

# # Get the active worksheet
# worksheet = workbook.active

# # Iterate over each row in the DataFrame
# for index, row in df.iterrows():
#     # Check if the article has been downloaded already
#     if row['is_downloaded'] == 1:
#         continue

#     # Get the hyperlink from Column B
#     cell = worksheet.cell(row=index+1, column=2)
#     hyperlink = cell.hyperlink

#     # If there is no hyperlink, skip this row
#     if hyperlink is None:
#         continue

#     # Extract the URL of the article from the hyperlink
#     article_url = hyperlink.target

#     # Use BeautifulSoup to extract the PDF link
    
#     pdf_link = find_pdf_links(article_url)
#     print(index, article_url, '___', pdf_link)
#     # response = requests.get(article_url)
#     # soup = BeautifulSoup(response.content, 'html.parser')
#     # pdf_link = soup.find('a', href=lambda href: href and (href.endswith('.pdf') or ('pdf' in str(href).lower())))
#     if pdf_link is None:
#         pdf_link = soup.find('a', text=lambda text: text and ('pdf' in text.lower() or 'download' in text.lower()))

#     # If a PDF link is found, download the file and mark the row as downloaded
#     if pdf_link is not None:
#         pdf_url = pdf_link['href']
#         if not pdf_url.startswith('http'):
#             pdf_url = article_url + pdf_url if pdf_url.startswith('/') else article_url + '/' + pdf_url
#         response = requests.get(pdf_url)

#         with open(f"article_{index}.pdf", 'wb') as f:
#             f.write(response.content)

#         # Update the DataFrame to mark the row as downloaded
#         df.at[index, 'is_downloaded'] = 1
        
#     asdfasdf

# # Save the updated DataFrame to a new spreadsheet
# df.to_excel('articles_downloaded.xlsx', index=False)

In [ ]:
# Load the spreadsheet into a pandas DataFrame
df = pd.read_excel(sheet_name)

# Load the Excel workbook using openpyxl
workbook = load_workbook(sheet_name)

# Get the active worksheet
worksheet = workbook.active

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the article has been downloaded already
    if row['is_downloaded'] == 1:
        continue

    # Get the hyperlink from Column B
    cell = worksheet.cell(row=index+1, column=2)
    hyperlink = cell.hyperlink

    # If there is no hyperlink, skip this row
    if hyperlink is None:
        continue

    # Extract the URL of the article from the hyperlink
    article_url = hyperlink.target

    # Use BeautifulSoup to extract the PDF link
    pdf_url, method, current_url = find_pdf_links(article_url)
    print(index, article_url, '___', pdf_url, method, current_url)

    if pdf_url is None:
        response = requests.get(article_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_link = soup.find('a', href=lambda href: href and (href.endswith('.pdf') or ('pdf' in str(href).lower())))

        if pdf_link is None:
            pdf_link = soup.find('a', text=lambda text: text and ('pdf' in text.lower() or 'download' in text.lower()))

        if pdf_link is not None:
            pdf_url = pdf_link['href']

    # If a PDF link is found, download the file and mark the row as downloaded
    if pdf_url is not None:
        if not pdf_url.startswith('http'):
            pdf_url = article_url + pdf_url if pdf_url.startswith('/') else article_url + '/' + pdf_url
        response = requests.get(pdf_url)

        with open(f"article_{index}.pdf", 'wb') as f:
            f.write(response.content)

        # Update the DataFrame to mark the row as downloaded
        df.at[index, 'is_downloaded'] = 1

    # Save the updated DataFrame to a new spreadsheet
    df.to_excel('articles_downloaded.xlsx', index=False)

    # Sleep for 10 seconds to avoid overwhelming the server
    time.sleep(10)


In [ ]:


# Set the name of the Excel sheet



In [ ]:
def find_pdf_links(article_url):
    # Create a Chrome webdriver
    driver = webdriver.Chrome()

    # Load the article URL
    driver.get(article_url)

    try:
        # Find all buttons and links that contain the word "pdf"
        pdf_buttons = driver.find_elements(By.XPATH, "//button[contains(text(),'PDF')] | //a[contains(text(),'PDF')]")

        # If no buttons or links are found, return None
        if len(pdf_buttons) == 0:
            return None, None, driver.current_url

        # Print all the buttons and links that contain the word "pdf"
        for pdf_button in pdf_buttons:
            print(pdf_button.text)

            # Try to click the PDF button using different methods
            method = None
            try:
                pdf_button.click()
                method = "click()"
            except:
                pass

            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('xpath')))).click()
                method = "xpath"
            except:
                pass

            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('href')))).click()
                method = "href"
            except:
                pass

            try:
                driver.execute_script("arguments[0].click();", pdf_button)
                method = "execute_script"
            except:
                pass

            # Wait for 10 seconds to see if the page redirects
            time.sleep(10)

            # Check if the current URL is the same as the article URL
            if driver.current_url == article_url:
                continue
            else:
                return driver.current_url, method,driver.current_url

    finally:
        # Close the webdriver
        driver.quit()

    # If no PDF link is found, return None
    return None, None,driver.current_url


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import load_workbook
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import PyPDF2

sheet_name = '/Users/phil/Downloads/dissertation REFS.xlsx'


In [ ]:
def find_pdf_links(driver, article_url):
    """
    Uses Selenium and Chrome to navigate to the article URL and extract the URL of the PDF file.

    Parameters:
        driver (webdriver.Chrome): The Chrome webdriver instance to use for navigation.
        article_url (str): The URL of the article to download.

    Returns:
        Tuple containing the PDF URL, method used to find it, and the current URL of the driver after the method is used.
        Returns (None, None, None) if a PDF link is not found.
    """
    driver.get(article_url)
    current_url = driver.current_url

    # Find the PDF link using a variety of methods
    try:
        # Wait for the "Download PDF" button to become visible
        download_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'Download PDF')]"))
        )
        download_button.click()
        method = 'download_button'

    except:
        try:
            # Find the PDF link using a text-based search
            pdf_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(text(),'PDF')]"))
            )
            pdf_url = pdf_link.get_attribute('href')
            method = 'text_search'

        except:
            try:
                # Find the PDF link using an attribute-based search
                pdf_link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[@href and contains(@href,'.pdf')]"))
                )
                pdf_url = pdf_link.get_attribute('href')
                method = 'attribute_search'

            except:
                # If no PDF link is found, return None
                return None, None, driver.current_url

        # If a PDF link is found, return it
        return pdf_url, method, driver.current_url

    return driver.current_url, method, driver.current_url


def validate_pdf(filename):
    try:
        with open(filename, 'rb') as f:
            pdf_reader = PyPDF2.PdfFileReader(f)
            num_pages = pdf_reader.getNumPages()
            return num_pages > 0
    except:
        return False

In [ ]:
import io
import PyPDF2

# def is_pdf_malformed(file_path):
#     try:
#         with open(file_path, 'rb') as file:
#             pdf_reader = PyPDF2.PdfReader(file)
#         return False  # The PDF could be read successfully
#     except Exception as e:
#         # Check if the exception message contains the specific error related to malformed PDFs
#         if "malformed" in str(e):
#             return True  # The PDF is malformed
#     return False  # The PDF is not malformed


def validate_not_malformed_pdfs(file_paths):

    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
    except Exception as e:
        # Check if the exception message contains the specific error related to malformed PDFs
        if "Could not read malformed PDF file" in str(e):
            return False
    return True
    
        



def validate_pdf(filename):
    try:
        with open(filename, 'rb') as f:
            pdf_reader = PyPDF2.PdfFileReader(f)
            if pdf_reader.isEncrypted:
                pdf_reader.decrypt('')
            page = pdf_reader.getPage(0)
            page_content = page.extractText()
            if len(page_content) > 0:
                return True
            else:
                return False
    except:
        return False

# pdfs are mostly corrupt for somereason and I cant identiy using python IDO why 
# i get this error 
# Adobe Acrobat Reader could not open 'article_1.pdf' because it is either not a supported file type or because the file has been damaged (for example, it was sent as an email attachment and wasn't correctly decoded).
validate_not_malformed_pdfs("/Users/phil/Desktop/Phils_python_code/article_3.pdf"), validate_not_malformed_pdfs("/Users/phil/Desktop/Phils_python_code/article_1.pdf")

In [ ]:

# Start the Chrome driver and navigate to the USC library login page
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://libproxy.usc.edu/login?url=%27+location.href')

# Wait for user input to continue
input('Please login and press Enter to continue...')

# Load the spreadsheet into a pandas DataFrame

df = pd.read_excel(sheet_name)

# Load the Excel workbook using openpyxl
workbook = load_workbook(sheet_name)

# Get the active worksheet
worksheet = workbook.active

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the article has been downloaded already
    if row['is_downloaded'] == 1:
        continue

    # Get the hyperlink from Column B
    cell = worksheet.cell(row=index+1, column=2)
    hyperlink = cell.hyperlink

    # If there is no hyperlink, skip this row
    if hyperlink is None:
        continue

    # Extract the URL of the article from the hyperlink
    article_url = hyperlink.target

    # Use BeautifulSoup to extract the PDF link
    pdf_url, method, current_url = find_pdf_links(driver, article_url)
    print(index, article_url, '___', pdf_url, method, current_url)

    if pdf_url is None:
        response = requests.get(article_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_link = soup.find('a', href=lambda href: href and (href.endswith('.pdf') or ('pdf' in str(href).lower())))

        if pdf_link is None:
            pdf_link = soup.find('a', text=lambda text: text and ('pdf' in text.lower() or 'download' in text.lower()))

        if pdf_link is not None:
            pdf_url = pdf_link['href']

    # If a PDF link is found, download the file and mark the row as downloaded
    if pdf_url is not None:
        if not pdf_url.startswith('http'):
            pdf_url = article_url + pdf_url if pdf_url.startswith('/') else article_url + '/' + pdf_url
        response = requests.get(pdf_url)

        with open(f"article_{index}.pdf", 'wb') as f:
            f.write(response.content)

        # Update the DataFrame to mark the row as downloaded
        df.at[index, 'is_downloaded'] = 1

    # Save the updated DataFrame to a new spreadsheet
    df.to_excel('articles_downloaded.xlsx', index=False)

    # Sleep for 10 seconds to avoid overwhelming the server
    time.sleep(10)

# Close the webdriver
driver.quit()


In [ ]:
# def find_pdf_links(article_url):
#     # Create a Chrome webdriver
#     driver = webdriver.Chrome()

#     # Load the article URL
#     driver.get(article_url)

#     try:
#         # Find all buttons and links that contain the word "pdf"
#         pdf_buttons = driver.find_elements(By.XPATH, "//button[contains(text(),'PDF')] | //a[contains(text(),'PDF')]")

#         # If no buttons or links are found, return None
#         if len(pdf_buttons) == 0:
#             return None

#         # Print all the buttons and links that contain the word "pdf"
#         print(pdf_buttons)
        
#         for pdf_button in pdf_buttons:
#             print(pdf_button.text)
#             asdf

#             # Try to click the PDF button using different methods
#             try:
#                 pdf_button.click()
#             except:
#                 pass

#             try:
#                 WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('xpath')))).click()
#             except:
#                 pass

#             try:
#                 WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('href')))).click()
#             except:
#                 pass

#             try:
#                 driver.execute_script("arguments[0].click();", pdf_button)
#             except:
#                 pass
            
#             # Wait for the PDF link to become visible
#             pdf_link = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.XPATH, "//a[contains(text(),'PDF')]"))
#             )

#             # Extract the URL of the PDF file
#             pdf_url = pdf_link.get_attribute('href')

#             # If a PDF link is found, return it
#             if pdf_url is not None:
#                 return pdf_url

#     finally:
#         # Close the webdriver
#         driver.quit()

#     # If no PDF link is found, return None
#     return None


In [ ]:
# def find_pdf_links(article_url):
#     # Create a Chrome webdriver
#     driver = webdriver.Chrome()

#     # Load the article URL
#     driver.get(article_url)

#     try:
#         # Find all buttons and links that contain the word "pdf"
#         pdf_buttons = driver.find_elements(By.XPATH, "//button[contains(text(),'PDF')] | //a[contains(text(),'PDF')]")

#         # If no buttons or links are found, return None
#         if len(pdf_buttons) == 0:
#             return None

#         # Print all the buttons and links that contain the word "pdf"
#         for pdf_button in pdf_buttons:
#             print(pdf_button.text)

#             # Try to click the PDF button using different methods
#             try:
#                 pdf_button.click()
#             except:
#                 pass

#             try:
#                 WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('xpath')))).click()
#             except:
#                 pass

#             try:
#                 WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pdf_button.get_attribute('href')))).click()
#             except:
#                 pass

#             try:
#                 driver.execute_script("arguments[0].click();", pdf_button)
#             except:
#                 pass

#             # Wait for 10 seconds to see if the page redirects
#             time.sleep(10)

#             # Check if the current URL is the same as the article URL
#             if driver.current_url == article_url:
#                 continue
#             else:
#                 return driver.current_url

#     finally:
#         # Close the webdriver
#         driver.quit()

#     # If no PDF link is found, return None
#     return None
